In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
np.random.seed(180)
random.seed(180)

#### [Natural Language Processing with Disaster Tweets Dataset](https://www.kaggle.com/competitions/nlp-getting-started/overview)

In [3]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

In [4]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df_train["target"].value_counts(normalize=True)

target
0    0.57034
1    0.42966
Name: proportion, dtype: float64

In [6]:
text_id = df_test["id"].tolist()
test_x = df_test["text"].tolist()

In [7]:
df_0 = df_train[df_train["target"] == 0]["text"].tolist()
df_1 = df_train[df_train["target"] == 1]["text"].tolist()

train_x = df_0 + df_1
train_y = [0]*len(df_0) + [1]*len(df_1)

In [8]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

c:\Users\yveem\.conda\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def prepare_data(train_x, train_y):
    data = {"text": train_x, "label": train_y}
    dataset = Dataset.from_dict(data)
    return dataset.train_test_split(test_size=0.2, seed=42)

In [10]:
def tokenize_function(examples, tokenizer):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [12]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
base_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

c:\Users\yveem\.conda\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yveem\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-ba

In [15]:
base_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [16]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "k_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

In [17]:
model = get_peft_model(base_model, lora_config)

In [18]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [19]:
dataset = prepare_data(train_x, train_y)
tokenized_dataset = dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

Map: 100%|██████████| 1523/1523 [00:00<00:00, 1677.27 examples/s]


In [20]:
training_args = TrainingArguments(
    output_dir="./lora_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    push_to_hub=False,
)

c:\Users\yveem\.conda\envs\nlp\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\yveem\AppData\Local\Temp\ipykernel_9932\2305446495.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                                  
 33%|███▎      | 381/1143 [07:57<11:46,  1.08it/s]

{'eval_loss': 0.49444326758384705, 'eval_accuracy': 0.7793827971109653, 'eval_precision': 0.78125, 'eval_recall': 0.6818181818181818, 'eval_f1': 0.7281553398058253, 'eval_runtime': 41.9109, 'eval_samples_per_second': 36.339, 'eval_steps_per_second': 1.145, 'epoch': 1.0}


 44%|████▎     | 500/1143 [10:11<13:03,  1.22s/it]  

{'loss': 0.5606, 'grad_norm': 1.0269794464111328, 'learning_rate': 1.1251093613298338e-05, 'epoch': 1.31}


                                                  
 67%|██████▋   | 762/1143 [15:45<05:32,  1.15it/s]

...

                                                   
100%|██████████| 1143/1143 [23:44<00:00,  1.06s/it]

{'eval_loss': 0.4356352388858795, 'eval_accuracy': 0.8063033486539725, 'eval_precision': 0.8288288288288288, 'eval_recall': 0.696969696969697, 'eval_f1': 0.757201646090535, 'eval_runtime': 34.9696, 'eval_samples_per_second': 43.552, 'eval_steps_per_second': 1.373, 'epoch': 3.0}


100%|██████████| 1143/1143 [23:45<00:00,  1.25s/it]

{'train_runtime': 1425.4455, 'train_samples_per_second': 12.817, 'train_steps_per_second': 0.802, 'train_loss': 0.4878929585400946, 'epoch': 3.0}


TrainOutput(global_step=1143, training_loss=0.4878929585400946, metrics={'train_runtime': 1425.4455, 'train_samples_per_second': 12.817, 'train_steps_per_second': 0.802, 'total_flos': 403352940915648.0, 'train_loss': 0.4878929585400946, 'epoch': 3.0})

In [22]:
model.save_pretrained("./lora_distilbert_model")

In [23]:
def predict_tweet(tweet, tokenizer, model):
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return pred

my_tweet = """
For those who have been waiting for this scene...

Now, a mass exodus of settlers from northern occupied Palestine have left their homes burning and are fleeing.
"""

prediction = predict_tweet(my_tweet, tokenizer, model)
print(f"The predicted label is: {prediction}")

The predicted label is: 1


In [24]:
preds = []
ids = []
for _, row in df_test.iterrows():
  tweet = row["text"]
  ids.append(row["id"])

  prediction = predict_tweet(tweet, tokenizer, model)
  preds.append(prediction)

In [25]:
df_submit = pd.DataFrame({"id": ids, "target": preds})

In [26]:
df_submit.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [27]:
df_submit.to_csv("./results/distilbert.csv", index=False)